In [11]:
import numpy as np
import torch
import time
import pandas as pd
import torch.nn as nn
import torch.nn.functional as F
import matplotlib.pyplot as plt
import sklearn.metrics as skm
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader, TensorDataset
from IPython import display
display.set_matplotlib_formats("svg")

C:\Users\Admin\AppData\Local\Temp\ipykernel_12844\2889536757.py:12: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats("svg")


In [12]:
data = pd.read_csv("../FFN/mnist_train_small.csv", header=None)

# extract labels (number ID's) and remove from data
labels = data.values[:, 0]  # Convert DataFrame to NumPy before slicing
data = data.values[:, 1:]

# normalize the data to a range of [0 1]
dataNorm = data/np.max(data)

In [13]:
# Step 1: convert to tensor
dataT = torch.tensor(dataNorm).float()
labelsT = torch.tensor(labels).long()

# Step 2: use scikitlearn to split the data
train_data, test_data, train_labels, test_labels = train_test_split(dataT, labelsT, test_size = .1)

# Step 3: convert into Pytorch Datasets
train_data = TensorDataset(train_data, train_labels)
test_data = TensorDataset(test_data, test_labels)

# Step 4: translate into dataloader objects
batchsize = 32
train_loader = DataLoader(train_data, batch_size = batchsize, shuffle = True, drop_last = True)
test_loader = DataLoader(test_data, batch_size = test_data.tensors[0].shape[0])

In [14]:
# creating a class for the model
def createTheMNISTNet():
    class mnistNet(nn.Module):
        def __init__(self):
            super().__init__()

            # input layer
            self.input = nn.Linear(784, 64)

            # hidden layer
            self.fc1 = nn.Linear(64, 32)
            self.fc2 = nn.Linear(32, 32)

            # output layer
            self.output = nn.Linear(32, 10)

        # forward pass
        def forward(self, x):
            x = F.relu(self.input(x))
            x = F.relu(self.fc1(x))
            x = F.relu(self.fc2(x))
            return self.output(x)

    # create the model instance
    net = mnistNet()

    # loss function
    lossfun = nn.CrossEntropyLoss()

    # optimizer
    optimizer = torch.optim.Adam(net.parameters(), lr = .01) # Adam performs way too good for this model therefore we used SGD

    return net, lossfun, optimizer

In [15]:
 # train the model

def trainTheModel():

    # start the timer
    timerInFunction = time.process_time()

    # training parameter
    numepochs = 10
    
    # create a new model
    net, lossfun, optimizer = createTheMNISTNet()
    
    trainAcc = []
    testAcc = []
    losses = torch.zeros(numepochs)
    
    # train the model
    for epochi in range(numepochs):
        
        # loop over training data batches
        batchAcc = []
        batchLoss = []
        for X, y in train_loader:

            # forward pass and loss
            yHat = net(X)
            loss = lossfun(yHat, y)
            
            # backprop
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            # loss from this batch
            batchLoss.append(loss.item())

            # compute training accuracy for this batch
            matches = torch.argmax(yHat, axis = 1) == y # booleans (false/true); selects the class with the highest predicted probability for each input.
            # The == operator compares the predicted class to the actual class, returning a Boolean tensor (True for correct predictions, False for incorrect).
            matchesNumeric = matches.float() # convert to numbers (0/1)
            accuracyPct = 100*torch.mean(matchesNumeric) # avg and * by 100
            batchAcc.append(accuracyPct)
            
        # now that we've trained through the batches, get avg training accuracy
        trainAcc.append(np.mean(batchAcc))

        # get avg losses over the batches
        losses[epochi] = np.mean(batchLoss)

        # test accuracy
        X, y = next(iter(test_loader))  # extract X, y from test dataloader
        yHat = net(X)

        # compute test accuracy
        testAcc.append(100 * torch.mean((torch.argmax(yHat, axis = 1) == y).float()))

        # Finally, report the epoch number, computation time, and accuracy
        comptime = time.process_time() - timerInFunction
        print(f"Epoch {epochi + 1}/{numepochs}, elapsed time: {comptime:.2f} sec, test accuracy {testAcc[-1]:.0f}%")
        
    # function output
    return trainAcc, testAcc, losses, net

In [16]:
trainAcc, testAcc, losses, net = trainTheModel()

Epoch 1/10, elapsed time: 5.42 sec, test accuracy 88%
Epoch 2/10, elapsed time: 10.75 sec, test accuracy 94%
Epoch 3/10, elapsed time: 16.06 sec, test accuracy 93%
Epoch 4/10, elapsed time: 21.55 sec, test accuracy 94%
Epoch 5/10, elapsed time: 26.83 sec, test accuracy 95%
Epoch 6/10, elapsed time: 32.19 sec, test accuracy 96%
Epoch 7/10, elapsed time: 37.47 sec, test accuracy 94%
Epoch 8/10, elapsed time: 42.80 sec, test accuracy 94%
Epoch 9/10, elapsed time: 48.02 sec, test accuracy 95%
Epoch 10/10, elapsed time: 53.56 sec, test accuracy 95%


In [18]:
# now run a second timer over repeated iterations

# Start the timer! (note the different variable name)
timerOutsideFunction = time.process_time()

for i in range(10):
  trainTheModel()

TotalExperimentTime = time.process_time() - timerOutsideFunction
print(f'\n\n\nTotal elapsed experiment time: {TotalExperimentTime/60:.2f} minutes')

Epoch 1/10, elapsed time: 4.94 sec, test accuracy 92%
Epoch 2/10, elapsed time: 10.02 sec, test accuracy 94%
Epoch 3/10, elapsed time: 15.30 sec, test accuracy 94%
Epoch 4/10, elapsed time: 20.27 sec, test accuracy 95%
Epoch 5/10, elapsed time: 25.34 sec, test accuracy 95%
Epoch 6/10, elapsed time: 30.56 sec, test accuracy 95%
Epoch 7/10, elapsed time: 35.73 sec, test accuracy 95%
Epoch 8/10, elapsed time: 41.08 sec, test accuracy 94%
Epoch 9/10, elapsed time: 46.28 sec, test accuracy 95%
Epoch 10/10, elapsed time: 51.47 sec, test accuracy 95%
Epoch 1/10, elapsed time: 4.84 sec, test accuracy 91%
Epoch 2/10, elapsed time: 10.14 sec, test accuracy 94%
Epoch 3/10, elapsed time: 15.38 sec, test accuracy 94%
Epoch 4/10, elapsed time: 20.47 sec, test accuracy 92%
Epoch 5/10, elapsed time: 25.67 sec, test accuracy 94%
Epoch 6/10, elapsed time: 30.88 sec, test accuracy 95%
Epoch 7/10, elapsed time: 36.12 sec, test accuracy 93%
Epoch 8/10, elapsed time: 41.97 sec, test accuracy 95%
Epoch 9/10,